<a href="https://colab.research.google.com/github/Pavansree14/Python/blob/main/tes1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e3d783f03a3e69293a24c9380f3a4fdaa2d89f4c567a4dd7a4ba86ca9e53ed19
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import Sparkcontext, Sparkconf

ImportError: cannot import name 'Sparkcontext' from 'pyspark' (/usr/local/lib/python3.10/dist-packages/pyspark/__init__.py)

In [3]:
from pyspark import Sparkcontext, Sparkconf

ImportError: cannot import name 'Sparkcontext' from 'pyspark' (/usr/local/lib/python3.10/dist-packages/pyspark/__init__.py)

In [4]:
from pyspark import SparkContext, SparkConf

In [5]:
conf = SparkConf().setAppName("testApp1").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [6]:
print(sc)

<SparkContext master=local[*] appName=testApp1>


In [7]:
import random
randomlist  = random.sample(range(0,40),10)
print(randomlist)

[19, 27, 4, 29, 26, 22, 8, 14, 17, 37]


In [8]:
rdd1 = sc.parallelize(randomlist, 4)

rdd1.collect()

[19, 27, 4, 29, 26, 22, 8, 14, 17, 37]

In [9]:
rdd1.getnumpartitions()

AttributeError: 'RDD' object has no attribute 'getnumpartitions'

In [10]:
rdd1.getNumPartitions()

4

In [11]:
print(rdd1.glom().collect())

[[19, 27], [4, 29], [26, 22], [8, 14, 17, 37]]


In [12]:
rdd_map = rdd1.map(lambda item: (item + 1)*3)
rdd_map.collect()

[60, 84, 15, 90, 81, 69, 27, 45, 54, 114]

In [13]:
print(rdd_map.glom().collect())

[[60, 84], [15, 90], [81, 69], [27, 45, 54, 114]]


In [17]:
rdd_filter = rdd1.filter(lambda x: x%3 == 0)
rdd_filter.collect()

[27]

In [18]:
(rdd_filter.glom().collect())

[[27], [], [], []]